<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2023/blob/main/notebooks/%D0%9F%D0%B5%D1%80%D1%81%D0%BE%D0%BD%D0%B0%D0%B6%D0%B8%20NER_%D1%87%D0%B0%D1%81%D1%82%D1%8C_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Просто запускаем все кнопки подряд

In [ ]:
# для очистки (препроцессинга) текста...
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
ru_stop_words = stopwords.words('russian') # давайте для русского сохранять в отдельную переменную
stop_words = stopwords.words('english')

# pymorphy
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
!pip install natasha

from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [ ]:
!pip install -U pip setuptools wheel --user
!pip install -U spacy
!python -m spacy download ru_core_news_sm

## Внимание! Запустите ТОЛЬКО русский или английский, чтобы они не съели друг друга

In [ ]:
# для русского языка
import spacy
nlp_ru = spacy.load('ru_core_news_sm')

def get_ru_lemmas(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = []
  for i in text_tokens:
    if i[0].isalpha() and i not in ru_stop_words:
      clean_tokens.append(i)
  ru_lemmatized = []
  for word in clean_tokens:
    result = morph.parse(word)
    most_probable_result = result[0]
    normal_form = most_probable_result.normal_form
    ru_lemmatized.append(normal_form)
  return ru_lemmatized

In [ ]:
# для английского запустите эту ячейку
import spacy
nlp = spacy.load("en_core_web_sm")

def get_eng_lemmas(text):
  text = text.lower()
  text_tokens = word_tokenize(text)
  clean_tokens = [] # чистим список токенов
  for i in text_tokens:
    if i[0].isalpha() and i not in stop_words:
      clean_tokens.append(i)
  spacy_token = []
  doc = nlp(' '.join(clean_tokens))
  for token in doc:
    if token.lemma_ != '-' and token.lemma_ != '.':
        spacy_token.append(token.lemma_)
  return spacy_token

# Важно! Ниже нужно изменить название текста на ваш

In [ ]:
with open('Dostoevsky_PrestuplenieINakazanie.txt', 'r', encoding='utf-8') as f: # впишите ваше название.txt вместо Достоевского
  text = f.read()

# NER natasha

In [ ]:
doc = Doc(text)

In [ ]:
doc.segment(segmenter)
print(doc.tokens[:5])
print(doc.sents[:5])

In [ ]:
doc.tag_ner(ner_tagger)
print(doc.spans[:5])

In [ ]:
for span in doc.spans:
  span.normalize(morph_vocab)
  print(span)
  print(span.text)
  print(span.type)
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

In [ ]:
# Важно!!!
# вы можете поменять 'PER' на 'ORG' или 'LOC', чтобы собрать огранизации и локации

persons = []
for span in doc.spans:
  if span.type == 'PER' and span.text not in persons: # для простоты в этой строке можно поменять только одно слово

  # еще! еще вам нужно извлечь, например, персон ИЛИ локации (все вместе), можно еще так:
  # if (span.type == 'PER' or span.type == 'LOC') and span.text not in persons:

    persons.append(span.text)
persons

# NER spacy

In [ ]:
doc = nlp_ru(text[:1000000])

In [ ]:
from spacy import displacy
displacy.render(list(doc.sents)[1], style='dep')

In [ ]:
for ent in doc.ents:
  print(ent.lemma_, '->', ent.label_)

In [ ]:
LOC = []
PER = []
ORG = []
for ent in doc.ents:
    if ent.label_ == 'LOC':
        if ent.lemma_ not in LOC:
            LOC.append(ent.lemma_)
    if ent.label_ == 'PER':
         if ent.lemma_ not in PER:
            PER.append(ent.lemma_)
    if ent.label_ == 'ORG':
        if ent.lemma_ not in ORG:
            ORG.append(ent.lemma_)

In [ ]:
PER # на самом деле, spaCy отдает вам и персон (PER), и организации (ORG), и локации (LOC)

# Сравниваем natasha и spaCy

In [ ]:
# я бы уже полученные NER от natasha лемматизировала... - это уберет лишние разночтения и глюки
person_natasha = []
for i in persons:
  person_natasha.append(' '.join(get_ru_lemmas(i)))
person_natasha

In [ ]:
# Важно!!!
# если у вас что-то сломалось, нужно посмотреть, что именно:
# person_natasha отсутствуют -> запускаем: res_person = list(set(PER)) вместо кода ниже
# list(set(PER)) отсутсвуют -> запускаем: res_person = set(person_natasha) вместо кода ниже
# а если вам по-прежнему нужны персоны ИЛИ локации (все вместе): res_person = list(set(PER) | set(LOC))
# + можно пересечь персон + локации из Наташи и спейси: res_person = list((set(PER) | set(LOC) & set(person_natasha))

# Код ниже ТОЛЬКО для тех, у кого оба списка нашли много персон

res_person = sorted(list(set(PER) & set(person_natasha))) # можно еще наташу не трогать, но PER поменять на ваше LOC или ORG
res_person

Что с ними делать дальше?

Можно автоматически собрать словарь, но все равно лишнее осталось

In [ ]:
dict_person = {}
for i in res_person:
  if i not in dict_person and ' ' not in i and '-то' not in i:
    dict_person[i] = [i]
  elif i not in dict_person and i[:i.find('-то')] not in dict_person and '-то' in i and i.find('-то') < i.find(' '):
    dict_person[i[:i.find('-то')]] = [i]
  elif i not in dict_person and i[:i.find('-то')] in dict_person and '-то' in i:
    dict_person[i[:i.find('-то')]] = dict_person[i[:i.find('-то')]] + [i]
  elif i not in dict_person and i[:i.find(' ')] not in dict_person and ' ' in i:
    dict_person[i[:i.find(' ')]] = [i]
  elif i not in dict_person and i[:i.find(' ')] in dict_person:
    dict_person[i[:i.find(' ')]] = dict_person[i[:i.find(' ')]] + [i]
  elif i not in dict_person and i[:i.rfind(' ')] in dict_person:
    dict_person[i[:i.find(' ')]] = dict_person[i[:i.rfind(' ')]] + [i]
dict_person

In [ ]:
for i in dict_person:
  if i not in dict_person[i]:
    dict_person[i] += [i]

dict_person

# Здесь придется чистить руками

сохраняем, ищем файл network.txt - чистим руками

In [ ]:
with open('network.txt', 'w', encoding='utf-8') as f:
  for i in dict_person:
    print(i, *dict_person[i], sep=';', file=f)

### Как чистить:

персона;персона;другое имя1;другое имя2....

станет:


персона : как мы ее назваем = персона, другое имя1, другое имя2...

Еще сразу пример про сеть терминов:

комсомол;разные варианты

Например:

комсомол;комсомол;союз молодежи;приводной ремень...

В этом случае комсомол;комсомол - это не ошибка: под словом комсомол мы имеем в виду имя точечки в сети, а также один из возможных вариантов, как его могут упомянуть в тексте